In [1]:
import numpy as np
import tensorflow as tf
import tensorflow.keras as keras

In [2]:
def generate_time_series(batch_size, n_steps):
    freq1, freq2, offsets1, offsets2 = np.random.rand(4, batch_size, 1)
    time = np.linspace(0, 1, n_steps)
    series = 0.5 * np.sin((time - offsets1) * (freq1 * 10 + 10)) # wave 1
    series += 0.2 * np.sin((time - offsets2) * (freq2 * 20 + 20)) # wave 2
    series += 0.1 * (np.random.rand(batch_size, n_steps) - 0.5) # noise
    return series[..., np.newaxis].astype(np.float32)    

In [3]:
n_steps = 50
series = generate_time_series(10000, n_steps + 1)
X_train, y_train = series[:7000, :n_steps], series[:7000, -1]
X_valid, y_valid = series[7000:9000, :n_steps], series[7000:9000, -1]
X_test, y_test = series[9000:, :n_steps], series[9000:, -1]

In [4]:
y_pred = X_valid[:, -1]
mse_baseline = np.mean(keras.losses.mean_squared_error(y_valid, y_pred))
print(mse_baseline)

0.020576663


In [5]:
model_linear = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[50, 1]),
    keras.layers.Dense(1)
])

model_linear.compile(optimizer='adam', loss='mean_squared_error', metrics=['mean_squared_error'])

model_linear.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 50)                0         
_________________________________________________________________
dense (Dense)                (None, 1)                 51        
Total params: 51
Trainable params: 51
Non-trainable params: 0
_________________________________________________________________


In [6]:
model_linear.fit(x=X_train, y=y_train, validation_data=(X_valid, y_valid), epochs=20, verbose=1)

Epoch 1/20
219/219 [==============================] - 2s 7ms/step - loss: 0.1888 - mean_squared_error: 0.1888 - val_loss: 0.0573 - val_mean_squared_error: 0.0573
Epoch 2/20
219/219 [==============================] - 1s 6ms/step - loss: 0.0384 - mean_squared_error: 0.0384 - val_loss: 0.0255 - val_mean_squared_error: 0.0255
Epoch 3/20
219/219 [==============================] - 1s 7ms/step - loss: 0.0205 - mean_squared_error: 0.0205 - val_loss: 0.0165 - val_mean_squared_error: 0.0165
Epoch 4/20
219/219 [==============================] - 1s 7ms/step - loss: 0.0147 - mean_squared_error: 0.0147 - val_loss: 0.0126 - val_mean_squared_error: 0.0126
Epoch 5/20
219/219 [==============================] - 1s 7ms/step - loss: 0.0118 - mean_squared_error: 0.0118 - val_loss: 0.0104 - val_mean_squared_error: 0.0104
Epoch 6/20
219/219 [==============================] - 1s 6ms/step - loss: 0.0101 - mean_squared_error: 0.0101 - val_loss: 0.0090 - val_mean_squared_error: 0.0090
Epoch 7/20
219/219 [========

In [7]:
model_RNN = keras.models.Sequential([
    keras.layers.SimpleRNN(20, return_sequences=True, input_shape=[None, 1]),
    keras.layers.SimpleRNN(20, return_sequences=True),
    keras.layers.SimpleRNN(1)
])

model_RNN.compile(optimizer='adam', loss='mean_squared_error', metrics=['mean_squared_error'])

model_RNN.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn (SimpleRNN)       (None, None, 20)          440       
_________________________________________________________________
simple_rnn_1 (SimpleRNN)     (None, None, 20)          820       
_________________________________________________________________
simple_rnn_2 (SimpleRNN)     (None, 1)                 22        
Total params: 1,282
Trainable params: 1,282
Non-trainable params: 0
_________________________________________________________________


In [8]:
model_RNN.fit(x=X_train, y=y_train, validation_data=(X_valid, y_valid), epochs=1, verbose=1)

219/219 [==============================] - 99s 450ms/step - loss: 0.0494 - mean_squared_error: 0.0494 - val_loss: 0.0113 - val_mean_squared_error: 0.0113


In [11]:
series = generate_time_series(1, n_steps + 10)
X_new, Y_new = series[:, :n_steps], series[:, n_steps:]
X = X_new
for step_ahead in range(10):
    y_pred_one = model_RNN.predict(X[:, step_ahead:])[:, np.newaxis, :]
    X = np.concatenate([X, y_pred_one], axis=1)

Y_pred = X[:, n_steps:]

print(Y_pred)

[[[ 0.02562159]
  [ 0.2714945 ]
  [ 0.45312944]
  [ 0.53903043]
  [ 0.5427865 ]
  [ 0.50290084]
  [ 0.41822007]
  [ 0.27325052]
  [ 0.11954144]
  [-0.012418  ]]]


In [12]:
series = generate_time_series(10000, n_steps + 10)
X_train, y_train = series[:7000, :n_steps], series[:7000, -10:, 0]
X_valid, y_valid = series[7000:9000, :n_steps], series[7000:9000, -10:, 0]
X_test, y_test = series[9000:, :n_steps], series[9000:, -10:, 0]

In [13]:
model_RNN = keras.models.Sequential([
    keras.layers.SimpleRNN(20, return_sequences=True, input_shape=[None, 1]),
    keras.layers.SimpleRNN(20),
    keras.layers.Dense(10)
])

model_RNN.compile(optimizer='adam', loss='mean_squared_error', metrics=['mean_squared_error'])

model_RNN.summary()

model_RNN.fit(x=X_train, y=y_train, validation_data=(X_valid, y_valid), epochs=10, verbose=1)

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn_3 (SimpleRNN)     (None, None, 20)          440       
_________________________________________________________________
simple_rnn_4 (SimpleRNN)     (None, 20)                820       
_________________________________________________________________
dense_1 (Dense)              (None, 10)                210       
Total params: 1,470
Trainable params: 1,470
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
219/219 [==============================] - 66s 303ms/step - loss: 0.0827 - mean_squared_error: 0.0827 - val_loss: 0.0377 - val_mean_squared_error: 0.0377
Epoch 2/10
219/219 [==============================] - 68s 309ms/step - loss: 0.0307 - mean_squared_error: 0.0307 - val_loss: 0.0246 - val_mean_squared_error: 0.0246
Epoch 3/10
219/219 [==============================] 

In [16]:
model_RNN.fit(x=X_train, y=y_train, validation_data=(X_valid, y_valid), epochs=10, verbose=1)

Epoch 1/10
219/219 [==============================] - 66s 303ms/step - loss: 0.0106 - mean_squared_error: 0.0106 - val_loss: 0.0094 - val_mean_squared_error: 0.0094
Epoch 2/10
219/219 [==============================] - 66s 300ms/step - loss: 0.0104 - mean_squared_error: 0.0104 - val_loss: 0.0150 - val_mean_squared_error: 0.0150
Epoch 3/10
219/219 [==============================] - 64s 290ms/step - loss: 0.0101 - mean_squared_error: 0.0101 - val_loss: 0.0107 - val_mean_squared_error: 0.0107
Epoch 4/10
219/219 [==============================] - 64s 294ms/step - loss: 0.0096 - mean_squared_error: 0.0096 - val_loss: 0.0091 - val_mean_squared_error: 0.0091
Epoch 5/10
219/219 [==============================] - 64s 291ms/step - loss: 0.0098 - mean_squared_error: 0.0098 - val_loss: 0.0090 - val_mean_squared_error: 0.0090
Epoch 6/10
219/219 [==============================] - 65s 295ms/step - loss: 0.0097 - mean_squared_error: 0.0097 - val_loss: 0.0111 - val_mean_squared_error: 0.0111
Epoch 7/10

In [15]:
Y_pred = model_RNN.predict(X_new)

print(Y_pred)

[[0.02828513 0.26666346 0.41556624 0.51363164 0.5398246  0.5215579
  0.37539163 0.31676355 0.26437658 0.17922427]]
